# Library

In [1]:
import os

from io import StringIO
import datetime
import pandas as pd

import psycopg2

from pprint import pprint 

# version info
print("""
Version:
      - Pandas  --  {}
      - Psycopg --  {}
""".format(pd.__version__, psycopg2.__version__))


Version:
      - Pandas  --  1.3.4
      - Psycopg --  2.8.6 (dt dec pq3 ext lo64)



# Connecting to PostgreSQL 

In [2]:
# Connection params
dbname = "postgres"
user = "postgres"
password = "***" #this is your password and information 
host = "localhost"
port = "5432" #this is the local port number


In [3]:
# --+ Basic interaction:
# create a new database session
connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)

# open a cursor to perform database commands and queries
cursor = connection.cursor()

# --+ Run some SQL command:
# get current date
cursor.execute("SELECT current_date;")
date = cursor.fetchone() # fetchone (returns a single tuple)
print("Current date: ", date[0].isoformat()) # print the info


# --+ Close the session:
# close the cursor
cursor.close()
# close the connection
connection.close()

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [ ]:
# Let's try to connect and print some info
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

# Homework 1

1. Create a new Database named smm695
2. Create a new Schema named homework1
3. Create a new Table with the following characteristics:

| column_name | data_type | length/precision |
|-------------|-----------|------------------|
| id          | serial    |                  |
| first_name  | varchar   | 20               |
| last_name   | varchar   | 20               |
| email       | varchar   | 80               |
| salary      | numeric   |                  |

4. Insert the following values:

| id | first_name | last_name | email                 | salary |
|----|------------|-----------|-----------------------|--------|
| 1  | Dave       | Alstom    | davealstom@google.com | 50000  |
| 2  | Hunter     | Reese     | reese1998@hotmail.nl  | 37000  |
| 3  | Kerys      | Mcclure   | mcclure@gmail.com     | 28000  |

5. Create a table that could contain the following values:

| id | name    | country        | postal_code | date_of_birth |
|----|---------|----------------|-------------|---------------|
| 1  | Martina | Italy          | 04929       | 1995-06-18    |
| 2  | Lenny   | United Kingdom | E2 9AD      | 1980-05-13    |
| 3  | Taha    | China          | 100023      | 1994-09-12    |

## Create a new Database

In [ ]:
# CREATE DATABASE
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)

    # Set autocommit
    connection.autocommit = True
    cursor = connection.cursor()

    # Create a new database
    cursor.execute("CREATE DATABASE smm695;")
    print(cursor.statusmessage)


except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

## CREATE SCHEMA and TABLES

In [ ]:
# Set the new db
dbname = 'smm695'

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Create a schema
    cursor.execute("CREATE SCHEMA homework1;")

    print(cursor.statusmessage, '\n')

    # Create a table storing employees data
    cursor.execute("""CREATE TABLE homework1.employees (
    id serial, 
    first_name  varchar(20),
    last_name varchar(20),
    email varchar(80),
    salary numeric);""")

    print(cursor.statusmessage, '\n')

    # Create a table storing people data

    second_table_sql = """CREATE TABLE homework1.person (
    id serial,
    name varchar(10),
    country varchar(20),
    postal_code text,
    dob date);"""

    cursor.execute(second_table_sql)

    print(cursor.statusmessage)

    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)
    
finally:
        if connection is not None:
            cursor.close()
            connection.close()

## Insert values: loop

In [ ]:
# data to insert
employees = """
first_name, last_name, email,                 salary
Dave,       Alstom,    davealstom@google.com, 50000
Hunter,     Reese,     reese1998@hotmail.nl,  37000
Kerys,      Mcclure,   mcclure@gmail.com,     28000
"""
# loading the database
db = pd.read_csv(StringIO(employees),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')

# obtaining tuples
values = list(zip(*map(db.get, db)))

# array of tuples 
values

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()
    
    # SQL INSERT
    insert_sql = """INSERT INTO homework1.employees (first_name, last_name, email, salary)
    VALUES (%s, %s, %s, %s)"""
    
    # Insert data
    for value in values:
        cursor.execute(insert_sql, value)
        print(cursor.statusmessage)

    # Commit changes
    connection.commit()

except  psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

## Inserting values: executemany

In [ ]:
values = [
('Martina',  'Italy', '04929',datetime.date(1995,6,19)),
('Lenny', 'United Kingdom',  'E2 9AD',  datetime.date(1980,5,13)),
('Taha', 'China', '100023',   datetime.date(1994,9,12))]

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()
    
    # SQL INSERT
    insert_sql = """INSERT INTO homework1.person (name, country, postal_code, dob)
    VALUES (%s, %s, %s, %s);"""
    
    # Insert data
    cursor.executemany(insert_sql, values)

    # Commit changes
    connection.commit()


except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

# Homework 2

1. Create a schema named homework 2 
2. Create a table with the following characteristics:

| column_name   | data_type | length/precision |
|---------------|-----------|------------------|
| id            | serial    |                  |
| name          | varchar   | 20               |
| department_id | int       |                  |
| phone_number  | varchar   | 20               |
| salary        | numeric   |                  |

3. Apply the following constraints:

| column        | constraints                                  |
|---------------|----------------------------------------------|
| id            | PRIMARY KEY                                  |
| name          | NOT NULL                                     |
| department_id | CHECK(department_id = 1 OR department_id =2) |
| phone_number  | UNIQUE                                       |
| salary        | NOT NULL                                     |

4. Insert the following observations:

| id | name     | department_id | phone_number | salary |
|----|----------|---------------|--------------|--------|
| 1  | John     | 2             | 690.623.6568 | 20000  |
| 2  | Leo      | 1             | 690.623.6708 | 80000  |
| 3  | Diana    | 1             | 690.623.0007 | 110000 |
| 4  | Paula    | 2             | 690.623.6500 | 35000  |
| 5  | Simon    | 2             | 690.623.9834 | 45000  |
| 6  | Jennifer | 2             |              | 18000  |

5. Calculate the average salary by department

 | department_id | avg_salary | 
 |---------------|------------|
 | 1             | 95000.00   |
 | 2             | 29500.00   |

6. Calculate the maximum salary by department

 | department_id | avg_salary |
 |---------------|------------|
 | 1             | 110000     |
 | 2             | 45000      |

7. Count the number of observations that contain the letter 'o' in the 'name' field

| count_o |
|---------|
| 3       |

8. Import location.csv and car.csv

9. Count the number of female observations with _`dob'_ between 1990 and 1992

| count |
|-------|
| 112   |

10. Select the two car models with the highest price for the last ten years

 | car_model | max    |
 |-----------|--------|
 | Caliber   | 999718 |
 | MKT       | 995884 |

## Create Schema and Table

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Create a schema
    cursor.execute("CREATE SCHEMA homework2;")

    print(cursor.statusmessage, '\n')

    # Create a table storing employees data
    cursor.execute("""CREATE TABLE homework2.employee (
    id serial PRIMARY KEY,
    name varchar(20) NOT NULL,
    department_id int,
    phone_number varchar(20) UNIQUE,
    salary numeric NOT NULL,
    CHECK (department_id = 1 OR department_id = 2));""")

    print(cursor.statusmessage, '\n')

    # Commit changes
    connection.commit()


except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

## Insert values: dictionary

In [ ]:
employees = """
name,     department_id, phone_number, salary
John,     2,            690.623.6568,  20000
Leo,      1,            690.623.6708,  80000
Diana,    1,            690.623.0007,  110000
Paula,    2,            690.623.6500,  35000
Simon,    2,            690.623.9834,  45000
Jennifer, 2,                        ,  18000
"""

# loading the database
db = pd.read_csv(StringIO(employees),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')

# data to dictionary
array_dict = db.to_dict('records')

pprint(array_dict)

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()
    
    # SQL INSERT
    insert_sql = """INSERT INTO homework2.employee (name, department_id, phone_number, salary)
    VALUES (%(name)s, %(department_id)s, %(phone_number)s, %(salary)s)"""
    
    # Insert data:
    cursor.executemany(insert_sql, array_dict)

    # Commit changes
    connection.commit()


except  psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

## Tasks 5 to 7

In [ ]:
#Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # 5. Calculate the average salary by department
    cursor.execute("""
    SELECT department_id, ROUND(AVG(salary),2) AS avg_salary  FROM homework2.employee 
    GROUP BY department_id ORDER BY department_id;
    """)
    five = cursor.fetchall()

    # 6. Calculate the maximum salary by department
    cursor.execute("""
    SELECT department_id, MAX(salary) AS max_salary  FROM homework2.employee 
    GROUP BY department_id ORDER BY department_id;
    """)
    six = cursor.fetchall()
    
    # 7. Count the number of observations that contain the letter 'o' in the 'name' field
    cursor.execute("""
    SELECT COUNT(*) AS count_o  FROM homework2.employee 
    WHERE name LIKE '%o%';
    """)
    seven = cursor.fetchone()
    
    # Show results
    print("""
    Average salary by department: 
        {}
    
    Maximum salary by department: 
        {}
    
    Observations that contain the letter o: 
        {}
    """.format(five, six, seven[0]))
    

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

## Import csv

In [ ]:
PATH = os.path.dirname(os.getcwd())
FOLDER = 'data'

In [5]:
dbname = 'mydb'

# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Create car table
    cursor.execute("""
    CREATE TABLE people.car (
    id bigserial PRIMARY KEY,
    car_make varchar(50),
    car_model varchar(50),
    car_year int,
    price numeric);
    """)
    print(cursor.statusmessage, '\n')

    # Create location table
    cursor.execute("""
    CREATE TABLE people.location (
    id bigserial PRIMARY KEY,
    country varchar(50),
    city varchar(50),
    street_name varchar(50),
    street_number int,
    postal_code varchar(50));
    """)
    print(cursor.statusmessage, '\n')

    # Copy car.csv
    car = open(os.path.join(PATH, FOLDER, 'car.csv'), 'r') #file-like object whta is the issue if the data is too big
    copy_car = """
           COPY people.car(car_make, car_model, car_year, price) FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
    cursor.copy_expert(copy_car, file=car)
    
    cursor.execute("""SELECT * FROM people.car;""")
    car_fetch = cursor.fetchmany(2)
    print( 'Car:', [car_fetch[i] for i in range(0, len(car_fetch), 1)], '\n')

    # Copy location.csv
    location = open(os.path.join(PATH, FOLDER, 'location.csv'), 'r') #file-like object whta is the issue if the data is too big
    # 建議不要這樣用 所以會建立一個df然後當成被用
    copy_location = """
           COPY people.location(country, city, street_name, street_number, postal_code) FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
    cursor.copy_expert(copy_location, file=location)
    
    cursor.execute("""SELECT * FROM people.location;""")
    location_fetch = cursor.fetchmany(2)
    print( 'Location:', [location_fetch[i] for i in range(0, len(location_fetch), 1)])

    # Commit changes
    connection.commit()


except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()

Warning! could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?



NameError: name 'connection' is not defined

## Task 9 and 10

In [ ]:
# Connect to smm695 and perform some tasks
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # 9. Count the number of female observations with `dob' between 1990 and 1992
    cursor.execute("""
    SELECT COUNT(*) FROM people.person  
    WHERE gender = 'Female' AND EXTRACT(YEAR FROM dob) BETWEEN 1990 AND 1992;
    """)
    nine = cursor.fetchall()

    # 10. Select the two car models with the highest price for the last ten years
    cursor.execute("""
    SELECT car_model, MAX(price) FROM people.car 
    WHERE car_year BETWEEN 2010 AND 2020 
    GROUP BY car_model 
    ORDER BY MAX(price) DESC LIMIT 2;
    """)
    ten = cursor.fetchall()

    # Show results
    print("""
    Count: {}
    
    Carn models: 
        1. {}, $ {}
        2. {}, $ {}
    """.format(nine[0][0], ten[0][0], ten[0][1], ten[1][0], ten[1][1]))


except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()